<h3>Loading the initial data</h3>

In [146]:
import pandas as pd
import numpy as np

In [147]:
ds = pd.read_excel('ml_project1_data.xlsx')
ds.set_index('ID', inplace=True)

<h3>Brief exploration</h3>

In [148]:
print(ds.info(), "\nResponse rate: {}%".format(round(100*ds["Response"].mean(), 2)))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2240 entries, 6862 to 8720
Data columns (total 28 columns):
Year_Birth             2240 non-null int64
Education              2240 non-null object
Marital_Status         2240 non-null object
Income                 2216 non-null float64
Kidhome                2240 non-null int64
Teenhome               2240 non-null int64
Dt_Customer            2240 non-null object
Recency                2240 non-null int64
MntWines               2240 non-null int64
MntFruits              2240 non-null int64
MntMeatProducts        2240 non-null int64
MntFishProducts        2240 non-null int64
MntSweetProducts       2240 non-null int64
MntGoldProds           2240 non-null int64
NumDealsPurchases      2240 non-null int64
NumWebPurchases        2240 non-null int64
NumCatalogPurchases    2240 non-null int64
NumStorePurchases      2240 non-null int64
NumWebVisitsMonth      2240 non-null int64
AcceptedCmp3           2240 non-null int64
AcceptedCmp4           22

How many rows can we delete?

In [149]:
len(ds)*0.03

67.2

Getting the indexes (idx) for categorical/dummy and numerical variables:

In [150]:
ds.columns

Index(['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response'],
      dtype='object')

In [154]:
cat_or_dummy_vars=['Education','Marital_Status','Kidhome','Teenhome','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5',
                'AcceptedCmp1', 'AcceptedCmp2','Complain' ,'Response'] 

numerical_var=['Year_Birth','Income','Dt_Customer','Recency','MntWines','MntFruits','MntMeatProducts','MntFishProducts',
              'MntSweetProducts','MntGoldProds','NumDealsPurchases','NumWebPurchases','NumCatalogPurchases','NumStorePurchases',
              'NumWebVisitsMonth']

### Fixing datatype consistencies:

In [152]:
#Turning dates into date type:
ds.Dt_Customer=pd.to_datetime(ds['Dt_Customer'].str.replace('-',''),format='%Y%m%d',errors='ignore')

<h3>Finding duplicated rows</h3>

With the target variable

In [5]:
np.sum(ds.duplicated())

182

In [6]:
ds[ds.duplicated(keep=False)].sort_values(ds.columns.tolist())

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
ID,,,,,,,,,,,,,,,,,,,,,
2968,1943,PhD,Divorced,48948.0,0,0,2013-02-01,53,437,8,...,6,1,0,0,0,0,0,3,11,1
8800,1943,PhD,Divorced,48948.0,0,0,2013-02-01,53,437,8,...,6,1,0,0,0,0,0,3,11,1
4939,1946,Graduation,Together,37760.0,0,0,2012-08-31,20,84,5,...,7,0,0,0,0,0,0,3,11,0
8652,1946,Graduation,Together,37760.0,0,0,2012-08-31,20,84,5,...,7,0,0,0,0,0,0,3,11,0
10562,1946,Master,Married,66835.0,0,0,2013-09-28,21,620,26,...,2,0,0,0,0,0,0,3,11,0
7881,1946,Master,Married,66835.0,0,0,2013-09-28,21,620,26,...,2,0,0,0,0,0,0,3,11,0
455,1946,PhD,Married,51012.0,0,0,2013-04-18,86,102,9,...,6,0,0,0,0,0,0,3,11,0
7592,1946,PhD,Married,51012.0,0,0,2013-04-18,86,102,9,...,6,0,0,0,0,0,0,3,11,0
2114,1946,PhD,Single,82800.0,0,0,2012-11-24,23,1006,22,...,3,0,0,1,1,0,0,3,11,1


In [62]:
ds[ds.duplicated(keep=False)].groupby(ds.columns.tolist()).size().reset_index().rename(columns={0:'records'}).sort_values(['Dt_Customer'])

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,records
164,1988,Graduation,Single,68655.0,0,0,2012-08-10,95,456,19,...,0,0,0,0,0,0,3,11,0,2
114,1974,Graduation,Married,67445.0,0,1,2012-08-12,63,757,80,...,0,0,0,0,0,0,3,11,0,3
130,1977,Graduation,Married,42014.0,1,0,2012-08-17,56,244,15,...,0,0,0,0,0,0,3,11,1,2
120,1975,Master,Single,46098.0,1,1,2012-08-18,86,57,0,...,0,0,0,0,0,0,3,11,0,2
106,1973,2n Cycle,Married,35688.0,2,1,2012-08-22,94,73,3,...,0,0,0,0,0,0,3,11,0,2
46,1961,Graduation,Married,60544.0,1,1,2012-08-25,92,201,2,...,0,0,0,0,0,0,3,11,0,2
156,1985,Graduation,Single,29760.0,1,0,2012-08-29,87,64,4,...,0,0,0,0,0,0,3,11,0,2
1,1946,Graduation,Together,37760.0,0,0,2012-08-31,20,84,5,...,0,0,0,0,0,0,3,11,0,2
165,1988,PhD,Married,67546.0,0,0,2012-08-31,90,864,134,...,0,0,0,0,0,0,3,11,0,2
41,1960,Graduation,Together,62807.0,0,1,2012-08-31,83,526,28,...,0,0,0,0,0,0,3,11,0,2


Without the target variable

In [8]:
np.sum(ds.drop(['Response'], axis=1).duplicated())

201

<h3>Exploring missing values</h3>

In [23]:
def missing_values_reporter(df):    
    na_count = df.isna().sum() 
    ser = na_count[na_count > 0]
    return pd.DataFrame({"N missings": ser, "p missings": ser.divide(df.shape[0])})

rm_df_miss = missing_values_reporter(ds)
rm_df_miss

,N missings,p missings
Income,24,0.010714


Only the <strong>Income</strong> feature has missing values.